In [1]:
import pytest
import numpy as np
import glob
import json
import jwst
from astropy.io import fits, ascii
from jwst.associations.asn_from_list import asn_from_list
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from jwst.pipeline import Detector1Pipeline, Spec2Pipeline, Spec3Pipeline, collect_pipeline_cfgs
from astropy.coordinates import Angle
from astropy.table import Table, vstack, unique
from astropy.stats import sigma_clip
from jwst.pipeline import calwebb_spec3
from jwst.associations.asn_from_list import asn_from_list
import matplotlib.pyplot as plt
import random
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from jwst.outlier_detection import outlier_detection
from ci_watson.artifactory_helpers import get_bigdata

In [2]:
Slitfile1 = get_bigdata('jwst_validation_notebooks',
                                     'validation_data',
                                     'calwebb_spec2',
                                    'spec2_miri_test',
                                    'miri_lrs_slit_pt_nod1_v2.3.fits')
 
Slitfile2 = get_bigdata('jwst_validation_notebooks',
                                     'validation_data',
                                     'calwebb_spec2',
                                    'spec2_miri_test',
                                    'miri_lrs_slit_pt_nod2_v2.3.fits')

In [3]:
det_nod1 = Detector1Pipeline.call(Slitfile1, config_file='configfiles/calwebb_detector1.cfg', output_dir='mirisimv2.3sim_correctdither/detector1/')
det_nod2 = Detector1Pipeline.call(Slitfile2, config_file='configfiles/calwebb_detector1.cfg', output_dir='mirisimv2.3sim_correctdither/detector1/')

2021-06-07 14:08:55,648 - stpipe - INFO - PARS-DETECTOR1PIPELINE parameters found: /grp/crds/cache/references/jwst/jwst_miri_pars-detector1pipeline_0001.asdf
2021-06-07 14:08:55,760 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2021-06-07 14:08:55,761 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2021-06-07 14:08:55,762 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2021-06-07 14:08:55,764 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2021-06-07 14:08:55,766 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2021-06-07 14:08:55,767 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2021-06-07 14:08:55,769 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2021-06-07 14:08:55,770 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2021-06-07 14:08:55,771 - stpipe.Detector1Pipeline.firstfram

2021-06-07 14:08:56,350 - stpipe.Detector1Pipeline - INFO - Prefetch for READNOISE reference file is '/grp/crds/cache/references/jwst/jwst_miri_readnoise_0057.fits'.
2021-06-07 14:08:56,407 - stpipe.Detector1Pipeline - INFO - Prefetch for REFPIX reference file is 'N/A'.
2021-06-07 14:08:56,408 - stpipe.Detector1Pipeline - INFO - Prefetch for RESET reference file is '/grp/crds/cache/references/jwst/jwst_miri_reset_0047.fits'.
2021-06-07 14:08:56,446 - stpipe.Detector1Pipeline - INFO - Prefetch for RSCD reference file is '/grp/crds/cache/references/jwst/jwst_miri_rscd_0010.fits'.
2021-06-07 14:08:56,490 - stpipe.Detector1Pipeline - INFO - Prefetch for SATURATION reference file is '/grp/crds/cache/references/jwst/jwst_miri_saturation_0026.fits'.
2021-06-07 14:08:56,537 - stpipe.Detector1Pipeline - INFO - Prefetch for SUPERBIAS reference file is 'N/A'.
2021-06-07 14:08:56,537 - stpipe.Detector1Pipeline - INFO - Prefetch for TRAPDENSITY reference file is 'N/A'.
2021-06-07 14:08:56,538 - stp

2021-06-07 14:10:18,647 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /grp/crds/cache/references/jwst/jwst_miri_linearity_0024.fits
2021-06-07 14:10:35,949 - stpipe.Detector1Pipeline.linearity - INFO - Saved model in mirisimv2.3sim_correctdither/detector1/miri_lrs_slit_pt_nod1_v2.3_linearity.fits
2021-06-07 14:10:35,949 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity done
2021-06-07 14:10:36,024 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd running with args (<RampModel(2, 10, 1024, 1032) from miri_lrs_slit_pt_nod1_v2.3_linearity.fits>,).
2021-06-07 14:10:36,026 - stpipe.Detector1Pipeline.rscd - INFO - Step rscd parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/kmurray/lrs_nOPS7-OTB-20191216_simulation', 'type': 'baselin

2021-06-07 14:27:47,408 - stpipe.Detector1Pipeline.ramp_fit - INFO - Saved model in mirisimv2.3sim_correctdither/detector1/miri_lrs_slit_pt_nod1_v2.3_0_ramp_fit.fits
2021-06-07 14:27:47,604 - stpipe.Detector1Pipeline.ramp_fit - INFO - Saved model in mirisimv2.3sim_correctdither/detector1/miri_lrs_slit_pt_nod1_v2.3_1_ramp_fit.fits
2021-06-07 14:27:47,605 - stpipe.Detector1Pipeline.ramp_fit - INFO - Step ramp_fit done
2021-06-07 14:27:47,684 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale running with args (<ImageModel(1024, 1032) from miri_lrs_slit_pt_nod1_v2.3_0_ramp_fit.fits>,).
2021-06-07 14:27:47,688 - stpipe.Detector1Pipeline.gain_scale - INFO - Step gain_scale parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': 'gain_scale', 'search_output_file': True, 'input_dir': '/Users/kmurray/lrs_nOPS7-OTB-20191216_s

2021-06-07 14:27:49,151 - stpipe.Detector1Pipeline - INFO - Prefetching reference files for dataset: 'miri_lrs_slit_pt_nod2_v2.3.fits' reftypes = ['dark', 'gain', 'linearity', 'mask', 'persat', 'readnoise', 'refpix', 'reset', 'rscd', 'saturation', 'superbias', 'trapdensity', 'trappars']
2021-06-07 14:27:49,363 - stpipe.Detector1Pipeline - INFO - Prefetch for DARK reference file is '/grp/crds/cache/references/jwst/jwst_miri_dark_0058.fits'.
2021-06-07 14:27:49,433 - stpipe.Detector1Pipeline - INFO - Prefetch for GAIN reference file is '/grp/crds/cache/references/jwst/jwst_miri_gain_0008.fits'.
2021-06-07 14:27:49,482 - stpipe.Detector1Pipeline - INFO - Prefetch for LINEARITY reference file is '/grp/crds/cache/references/jwst/jwst_miri_linearity_0024.fits'.
2021-06-07 14:27:49,554 - stpipe.Detector1Pipeline - INFO - Prefetch for MASK reference file is '/grp/crds/cache/references/jwst/jwst_miri_mask_0023.fits'.
2021-06-07 14:27:49,614 - stpipe.Detector1Pipeline - INFO - Prefetch for PERSA

2021-06-07 14:27:59,688 - stpipe.Detector1Pipeline.reset - INFO - Using RESET reference file /grp/crds/cache/references/jwst/jwst_miri_reset_0047.fits
2021-06-07 14:29:08,049 - stpipe.Detector1Pipeline.reset - INFO - Step reset done
2021-06-07 14:29:08,119 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity running with args (<RampModel(2, 10, 1024, 1032) from miri_lrs_slit_pt_nod2_v2.3_lastframe.fits>,).
2021-06-07 14:29:08,123 - stpipe.Detector1Pipeline.linearity - INFO - Step linearity parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '/Users/kmurray/lrs_nOPS7-OTB-20191216_simulation'}
2021-06-07 14:29:08,479 - stpipe.Detector1Pipeline.linearity - INFO - Using Linearity reference file /grp/crds/cache/references/jwst/jwst_miri_linearity_0024.fits
2021-06-07 14:29:23

2021-06-07 14:44:32,889 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using GAIN reference file: /grp/crds/cache/references/jwst/jwst_miri_gain_0008.fits
2021-06-07 14:44:32,942 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using algorithm = ols
2021-06-07 14:44:32,943 - stpipe.Detector1Pipeline.ramp_fit - INFO - Using weighting = optimal
2021-06-07 14:44:33,025 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of leading groups that are flagged as DO_NOT_USE: 1
2021-06-07 14:44:33,026 - stpipe.Detector1Pipeline.ramp_fit - INFO - MIRI dataset has all pixels in the final group flagged as DO_NOT_USE.
2021-06-07 14:44:50,613 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of groups per integration: 10
2021-06-07 14:44:50,613 - stpipe.Detector1Pipeline.ramp_fit - INFO - Number of integrations: 2
2021-06-07 14:44:50,768 - stpipe.Detector1Pipeline.ramp_fit - INFO - Saved model in mirisimv2.3sim_correctdither/detector1/miri_lrs_slit_pt_nod2_v2.3_0_ramp_fit.fits
2021-06-07 14:44:50,954 - s

In [4]:
spec1 = Spec2Pipeline.call(det_nod1, save_results=True)
spec2 = Spec2Pipeline.call(det_nod2, save_results=True)

2021-06-07 14:44:51,645 - stpipe - WARNING - /Users/kmurray/anaconda3/envs/jwst7.7.1/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

2021-06-07 14:44:51,835 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2021-06-07 14:44:51,836 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2021-06-07 14:44:51,838 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2021-06-07 14:44:51,839 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2021-06-07 14:44:51,840 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2021-06-07 14:44:51,841 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2d

2021-06-07 14:44:51,995 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'miri_lrs_slit_pt_nod1_v2.3_gain_scale.fits' reftypes = ['apcorr', 'area', 'barshadow', 'camera', 'collimator', 'cubepar', 'dflat', 'disperser', 'distortion', 'drizpars', 'extract1d', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'fringe', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'msaoper', 'ote', 'pathloss', 'photom', 'regions', 'resol', 'sflat', 'specwcs', 'wavecorr', 'wavelengthrange', 'wfssbkg']
2021-06-07 14:44:52,175 - stpipe.Spec2Pipeline - INFO - Prefetch for APCORR reference file is '/grp/crds/cache/references/jwst/jwst_miri_apcorr_0007.fits'.
2021-06-07 14:44:52,255 - stpipe.Spec2Pipeline - INFO - Prefetch for AREA reference file is 'N/A'.
2021-06-07 14:44:52,255 - stpipe.Spec2Pipeline - INFO - Prefetch for BARSHADOW reference file is 'N/A'.
2021-06-07 14:44:52,256 - stpipe.Spec2Pipeline - INFO - Prefetch for CAMERA reference file is 'N/A'.
2021-06-07 14:44:52,256 - stpipe.Spec

2021-06-07 14:44:54,647 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step skipped.
2021-06-07 14:44:54,648 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract done
2021-06-07 14:44:54,704 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging running with args (<ImageModel(1024, 1032) from miri_lrs_slit_pt_nod1_v2.3_gain_scale.fits>,).
2021-06-07 14:44:54,706 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': ''}
2021-06-07 14:44:54,707 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step skipped.
2021-06-07 14:44:54,707 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging done
2021-06-07 14:44:54,767 - stpipe.Spec2Pipeline.srctype - INFO - Step srctype running with args (<Image

2021-06-07 14:44:58,960 - stpipe.Spec2Pipeline.resample_spec - INFO - Drizpars reference file: /grp/crds/cache/references/jwst/jwst_miri_drizpars_0001.fits
2021-06-07 14:44:59,098 - stpipe.Spec2Pipeline.resample_spec - WARNING - /Users/kmurray/anaconda3/envs/jwst7.7.1/lib/python3.9/site-packages/jwst/resample/resample_utils.py:153: RuntimeWarning: var_rnoise array not available.  Setting drizzle weight map to 1
  warnings.warn("var_rnoise array not available.  Setting drizzle weight map to 1",

2021-06-07 14:44:59,102 - stpipe.Spec2Pipeline.resample_spec - INFO - Resampling slit None (387, 44)
2021-06-07 14:44:59,565 - stpipe.Spec2Pipeline.resample_spec - INFO - Drizzling (1024, 1032) --> (387, 44)
2021-06-07 14:44:59,585 - stpipe.Spec2Pipeline.resample_spec - INFO - Update S_REGION to POLYGON ICRS  179.999087909 -0.000075806 180.000404817 -0.000075806 180.000404817 0.000033658 179.999087909 0.000033658
2021-06-07 14:44:59,655 - stpipe.Spec2Pipeline.resample_spec - INFO - Saved model i

2021-06-07 14:45:00,777 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'miri_lrs_slit_pt_nod2_v2.3_gain_scale.fits' reftypes = ['apcorr', 'area', 'barshadow', 'camera', 'collimator', 'cubepar', 'dflat', 'disperser', 'distortion', 'drizpars', 'extract1d', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'fringe', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'msaoper', 'ote', 'pathloss', 'photom', 'regions', 'resol', 'sflat', 'specwcs', 'wavecorr', 'wavelengthrange', 'wfssbkg']
2021-06-07 14:45:00,821 - stpipe.Spec2Pipeline - INFO - Prefetch for APCORR reference file is '/grp/crds/cache/references/jwst/jwst_miri_apcorr_0007.fits'.
2021-06-07 14:45:00,852 - stpipe.Spec2Pipeline - INFO - Prefetch for AREA reference file is 'N/A'.
2021-06-07 14:45:00,853 - stpipe.Spec2Pipeline - INFO - Prefetch for BARSHADOW reference file is 'N/A'.
2021-06-07 14:45:00,853 - stpipe.Spec2Pipeline - INFO - Prefetch for CAMERA reference file is 'N/A'.
2021-06-07 14:45:00,854 - stpipe.Spec

2021-06-07 14:45:02,930 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step skipped.
2021-06-07 14:45:02,931 - stpipe.Spec2Pipeline.imprint_subtract - INFO - Step imprint_subtract done
2021-06-07 14:45:02,990 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging running with args (<ImageModel(1024, 1032) from miri_lrs_slit_pt_nod2_v2.3_gain_scale.fits>,).
2021-06-07 14:45:02,991 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': True, 'suffix': None, 'search_output_file': True, 'input_dir': ''}
2021-06-07 14:45:02,992 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step skipped.
2021-06-07 14:45:02,993 - stpipe.Spec2Pipeline.msa_flagging - INFO - Step msa_flagging done
2021-06-07 14:45:03,052 - stpipe.Spec2Pipeline.srctype - INFO - Step srctype running with args (<Image

2021-06-07 14:45:04,284 - stpipe.Spec2Pipeline.resample_spec - INFO - Drizpars reference file: /grp/crds/cache/references/jwst/jwst_miri_drizpars_0001.fits
2021-06-07 14:45:04,400 - stpipe.Spec2Pipeline.resample_spec - WARNING - /Users/kmurray/anaconda3/envs/jwst7.7.1/lib/python3.9/site-packages/jwst/resample/resample_utils.py:153: RuntimeWarning: var_rnoise array not available.  Setting drizzle weight map to 1
  warnings.warn("var_rnoise array not available.  Setting drizzle weight map to 1",

2021-06-07 14:45:04,404 - stpipe.Spec2Pipeline.resample_spec - INFO - Resampling slit None (387, 44)
2021-06-07 14:45:04,920 - stpipe.Spec2Pipeline.resample_spec - INFO - Drizzling (1024, 1032) --> (387, 44)
2021-06-07 14:45:04,941 - stpipe.Spec2Pipeline.resample_spec - INFO - Update S_REGION to POLYGON ICRS  179.999611282 -0.000032303 180.000928190 -0.000032303 180.000928190 0.000077161 179.999611282 0.000077161
2021-06-07 14:45:05,013 - stpipe.Spec2Pipeline.resample_spec - INFO - Saved model i

In [5]:
asn3_files = glob.glob('*_cal.fits')
print(asn3_files)
asn3 = asn_from_list(asn3_files, rule=DMS_Level3_Base,product_name='combine_dithers_exposures')

2021-06-07 14:45:05,837 - stpipe - WARNING - /Users/kmurray/anaconda3/envs/jwst7.7.1/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



['miri_lrs_slit_pt_nod2_v2_cal.fits', 'miri_lrs_slit_pt_nod1_v2_cal.fits']


In [6]:
with open('sp3-lrs-slit-test_asn.json', 'w') as fp:
    fp.write(asn3.dump()[1])

In [7]:
Spec3Pipeline.call('sp3-lrs-slit-test_asn.json', save_results=True)

2021-06-07 14:45:06,907 - stpipe.Spec3Pipeline - INFO - Spec3Pipeline instance created.
2021-06-07 14:45:06,908 - stpipe.Spec3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2021-06-07 14:45:06,910 - stpipe.Spec3Pipeline.master_background - INFO - MasterBackgroundStep instance created.
2021-06-07 14:45:06,912 - stpipe.Spec3Pipeline.mrs_imatch - INFO - MRSIMatchStep instance created.
2021-06-07 14:45:06,914 - stpipe.Spec3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2021-06-07 14:45:06,916 - stpipe.Spec3Pipeline.resample_spec - INFO - ResampleSpecStep instance created.
2021-06-07 14:45:06,918 - stpipe.Spec3Pipeline.cube_build - INFO - CubeBuildStep instance created.
2021-06-07 14:45:06,921 - stpipe.Spec3Pipeline.extract_1d - INFO - Extract1dStep instance created.
2021-06-07 14:45:06,923 - stpipe.Spec3Pipeline.combine_1d - INFO - Combine1dStep instance created.
2021-06-07 14:45:06,994 - stpipe.Spec3Pipeline - INFO - Step Spec3Pipeline running

2021-06-07 14:45:11,237 - stpipe.Spec3Pipeline.resample_spec - INFO - Drizpars reference file: /grp/crds/cache/references/jwst/jwst_miri_drizpars_0001.fits
2021-06-07 14:45:11,393 - stpipe.Spec3Pipeline.resample_spec - INFO - Blending metadata for combine_dithers_exposures
2021-06-07 14:45:11,935 - stpipe.Spec3Pipeline.resample_spec - INFO - Resampling slit (387, 5)
2021-06-07 14:45:12,445 - stpipe.Spec3Pipeline.resample_spec - INFO - Drizzling (1024, 1032) --> (387, 5)
2021-06-07 14:45:12,452 - stpipe.Spec3Pipeline.resample_spec - INFO - Resampling slit (387, 5)
2021-06-07 14:45:12,981 - stpipe.Spec3Pipeline.resample_spec - INFO - Drizzling (1024, 1032) --> (387, 5)
2021-06-07 14:45:12,989 - stpipe.Spec3Pipeline.resample_spec - INFO - Update S_REGION to POLYGON ICRS  73.636907547 0.000073100 73.637030041 0.000073100 73.637030041 0.000083282 73.636907547 0.000083282
2021-06-07 14:45:13,400 - stpipe.Spec3Pipeline.resample_spec - INFO - Saved model in combine_dithers_exposures_s2d.fits
2

In [8]:
outlier_detect1 = fits.open('combine_dithers_exposures_s2d.fits')

2021-06-07 14:45:14,043 - stpipe - WARNING - /Users/kmurray/anaconda3/envs/jwst7.7.1/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



In [9]:
print(outlier_detect1.meta.wcs.bounding_box[0][1]) #, outlier_detect.meta.wcs.bounding_box[0][0],outlier_detect.meta.wcs.bounding_box[1][1],outlier_detect.meta.wcs.bounding_box[1][0])
bbox_w = outlier_detect.meta.wcs.bounding_box[0][1] - outlier_detect.meta.wcs.bounding_box[0][0]
bbox_ht = outlier_detect.meta.wcs.bounding_box[1][1] - outlier_detect.meta.wcs.bounding_box[1][0]
left = outlier_detect.meta.wcs.bounding_box[0][0]
right = outlier_detect.meta.wcs.bounding_box[0][1]
down = outlier_detect.meta.wcs.bounding_box[1][0]
up = outlier_detect.meta.wcs.bounding_box[1][1]
print('Model bbox (nod 1)= {0} '.format(outlier_detect.meta.wcs.bounding_box))
print('Model: Height x width of bounding box (nod 1) = {0} x {1} pixels'.format(bbox_ht, bbox_w))

AttributeError: 'HDUList' object has no attribute 'meta'

In [ ]:
pixloc = []
for i in range(len(file_list)):
    pixloc.append([random.randint(10,1000),random.randint(10,1000)])
    pixloc.append([random.randint(10,1000),random.randint(10,1000)])
    pixloc.append([random.randint(10,1000),random.randint(10,1000)])
    pixloc.append([random.randint(10,1000),random.randint(10,1000)])
pixloc2 = np.array(pixloc)
print(pixloc2)